In [28]:
import cv2
import tqdm
import numpy as np
import matplotlib.pyplot as plt

from cv_functions import (
    Pi, InvPi, camera_matrix, shortest_distance_to_line, box3d, projectpoints, undistortImage,
    normalize2d, hest, rotation_matrix, triangulate,pest, checkerboard_points, estimateHomographies,
    estimate_b, estimateIntrinsics, estimateExtrinsics, estimateExtrinsics, calibratecamera, triangulate_nonlin,
    gaussian1DKernel, gaussianSmoothing, structureTensor, harrisMeasure, cornerDetector,
    DrawLine, test_points, fit_line, pca_line, distance_to_line, ransac_line, scaleSpaced,
    differenceOfGaussians, detectBlobs, transformIm, Fest_8point, unpack_cv2_matches, SampsonsDistance,
    ransac_F, homography_dist_approx, ransac_H, estHomographyRANSAC, warpImage,
    unwrap, calculate_disparity)

prettyprint = lambda arr, pres=2: print(np.array2string(arr, precision=pres, suppress_small=True))

CrossOp = lambda v: np.array([[    0, -v[2],  v[1]],
                              [ v[2],     0, -v[0]],
                              [-v[1],  v[0],     0]])


# Part 0

## excercise 1

In [29]:
K = camera_matrix(1400., (750, 520))

R = cv2.Rodrigues(np.array([0.2, 0.2, -0.1]))[0]
t = np.array([[-0.08, 0.01, 0.03]]).T

q = np.array([[-0.38, 0.1, 1.32]]).T

Pi(projectpoints(K, R, t, InvPi(q)))


array([[557.52555675],
       [383.75661509]])

## excercise 2

In [30]:
p1 = np.array([[ 1.45349587e+02, -1.12915131e-01, 1.91640565e+00, -6.08129962e-01],
               [1.05603820e+02, 5.62792554e-02, 1.79040110e+00, -2.32182177e-01]])
p2 = np.array([[ 1.3753556, -1.77072961, 2.94511795, 0.04032374],
               [0.30936653, 0.37172814, 1.44007577, -0.03173825]])


In [31]:
H = hest(p1, p2)
H / H[0, 0]


array([[ 1.        ,  1.18450189,  2.73494707],
       [ 1.28494954,  1.44207391,  1.03919164],
       [ 3.92261028, -2.06451   , -4.72549386]])

## excercise 3

In [32]:
Fest_8point

<function cv_functions.Fest_8point(q1, q2)>

## excercise 4

In [33]:
K = np.array([[300, 0, 840], [0, 300, 620], [0, 0, 1]], float)
k3 = -0.2
k5 = 0.01
k7 = -0.03

p = np.array([[400, 500., 1]]).T
radial_dist_factors = [k3, k5, k7]

q = np.linalg.inv(K) @ p
q_d = projectpoints(np.eye(3), Q=InvPi(q), radial_dist_factors=radial_dist_factors)
Pi(K @ q_d)

array([[742.81960823],
       [593.49625679]])

## excercise 5

In [ ]:
harrisMeasure

## excercise 6

In [34]:
data = np.load('Exam24/materials/harris.npy', allow_pickle=True).item()

a = data.pop('g*(I_x^2)')
b = data.pop('g*(I_y^2)')
c = data.pop('g*(I_x I_y)')
assert not data

k = 0.06
tau = 5.

det = a * b - c ** 2
tr = a + b
r = det - k * tr ** 2

c = (
    (r[1:-1, 1:-1] > tau) &
    (r[1:-1, 1:-1] > r[2:, 1:-1]) &
    (r[1:-1, 1:-1] > r[:-2, 1:-1]) &
    (r[1:-1, 1:-1] > r[1:-1, 2:]) &
    (r[1:-1, 1:-1] > r[1:-1, :-2]) &
    (r[1:-1, 1:-1] > r[2:, 2:]) &
    (r[1:-1, 1:-1] > r[:-2, :-2]) &
    (r[1:-1, 1:-1] > r[2:, :-2]) &
    (r[1:-1, 1:-1] > r[:-2, 2:])
)
c = np.pad(c, ((1, 1), (1, 1)))
np.where(c) 

(array([2]), array([3]))

In [35]:
r

array([[10.9484, 11.0936, 10.8734, 10.6226, 10.1624],
       [10.7104, 11.6746, 12.38  , 11.7166, 11.9926],
       [10.4324, 11.4216, 12.2894, 12.7506, 12.7164],
       [ 9.2844, 10.4834, 11.5136, 12.135 , 12.6104],
       [ 7.9566,  8.9254, 10.0526, 10.9664, 11.9816]])

## excercise 7

## excercise 8

## excercise 9

In [43]:
sift_data = np.load("Exam24/materials/sift_data.npy", allow_pickle=True).item()
kp1 = sift_data["kp1"]
des1 = sift_data["des1"]
kp2 = sift_data["kp2"]
des2 = sift_data["des2"]

des1 /= np.linalg.norm(des1, 1, axis=1, keepdims=True)
des2 /= np.linalg.norm(des2, 1, axis=1, keepdims=True)

des1 = np.sqrt(des1)
des2 = np.sqrt(des2)

des1 /= np.linalg.norm(des1, 2, axis=1, keepdims=True)
des2 /= np.linalg.norm(des2, 2, axis=1, keepdims=True)


In [44]:
# bf = cv2.BFMatcher(crossCheck=True)
# matches = bf.match(des1, des2)
# len(matches)

In [45]:
def rat_test(ps, p):
    dis = np.linalg.norm(ps - p[None,:], axis=1)
    idxs = np.argsort(dis)
    keep = dis[idxs[0]] / dis[idxs[1]] < 0.8
    return idxs[0] if keep else -1

matches12 = np.array([rat_test(des2, des1[i]) for i in range(2000)])
matches21 = np.array([rat_test(des1, des2[i]) for i in range(2000)])
# matches = [i for i in range(2000) if i == matches21[matches12[i]]]
# len(matches)

213

In [54]:
tmp1 = [(i, e) for i, e in enumerate(matches12) if e > -1]
tmp2 = [(e, i) for i, e in enumerate(matches21) if e > -1]

In [57]:
len(set(tmp1) & set(tmp2))

213

In [46]:
(matches12 > -1).sum(), (matches21 > -1).sum()

(331, 371)

# Part 10

## excercise 10

In [13]:
def checkerboard_points(n,m):
    Q = np.zeros(shape=(3, n*m))
    for i in range(n):
        for j in range(m):
            Q[:, i*m+j] = [i - (n-1)/2.0,
                        j - (m-1)/2.0,
                        0]
    return Q

def get_rgb(path):
    bgr_img = cv2.imread(path)
    b,g,r = cv2.split(bgr_img)       # get b,g,r
    image = cv2.merge([r,g,b])
    return image

In [14]:
images = []
for i in range(5):
    image = get_rgb(f"Exam24/materials/board{i}.jpg")
    # image = cv2.resize(image, (600, 400))
    images.append(image)


In [15]:
imgpoints = []
for img in images:
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (7, 10))
    if ret == True:
        imgpoints.append(corners)
        # Draw and display the corners
        # painted = cv2.drawChessboardCorners(img.copy(), (7, 10), corners,ret)
        # cv2.imshow('img',painted)
        # cv2.waitKey(400)


checkerboard = checkerboard_points(10,7)
objpoints = [np.float32(checkerboard.T)]

for i in range(5):
    ret, K_cv2, distortion_coefficients, rvecs, tvecs = cv2.calibrateCamera(
            objpoints, [imgpoints[i]], gray.shape[::-1], None, distCoeffs=np.array([0.,0,0,0]),
            # nimages=5
            # flags = cv2.CALIB_FIX_K1+cv2.CALIB_FIX_K2+cv2.CALIB_FIX_K3+cv2.CALIB_FIX_K4+
            # cv2.CALIB_FIX_K5+cv2.CALIB_FIX_K6+cv2.CALIB_ZERO_TANGENT_DIST
        )
    print(K_cv2[0,0])


763.4554355678482
2066.458526542327
1738.558526997851
1655.370124404771
367.8142037882513


## excercise 11

In [16]:
calibratecamera

<function cv_functions.calibratecamera(qs: list[numpy.ndarray], Q: numpy.ndarray, *, verbose: bool = False) -> tuple[numpy.ndarray, list[numpy.ndarray], list[numpy.ndarray]]>

## excercise 12

In [58]:
sift = cv2.SIFT_create()
bf = cv2.BFMatcher(crossCheck=True)

img1 = cv2.imread('Exam24/materials/im1.jpg')
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2 = cv2.imread('Exam24/materials/im2.jpg')
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

matches = bf.match(des1, des2)

H, inliners = ransac_H(
    *unpack_cv2_matches(kp1, kp2, matches),
    t_sq=3.84*3**2,
    max_iters=1600,
    return_match_filter=True
)
prettyprint(H / H[0, 0], 4)

H_inv = np.linalg.inv(H)
prettyprint(H_inv / H_inv[0, 0], 4)

[[   1.        0.0074 -383.5467]
 [   0.068     0.9721  -40.8378]
 [   0.0002    0.        0.8854]]
[[  1.      -0.0077 432.8447]
 [ -0.0773   1.0976  17.128 ]
 [ -0.0002   0.       1.1289]]


## excercise 13

In [18]:
Fest_8point

<function cv_functions.Fest_8point(q1, q2)>

## excercise 14

## excercise 15

In [19]:
K = np.array([[300, 0, 840], [0, 300, 620.0], [0, 0, 1]], float)
R1 = cv2.Rodrigues(np.array([-2.3, -0.7, 1.0]))[0]
t1 = np.array([[0.0, -1.0, 4.0]], float).T
R2 = cv2.Rodrigues(np.array([-0.6, 0.5, -0.9]))[0]
t2 = np.array([[0.0, 0.0, 9.0]], float).T
R3 = cv2.Rodrigues(np.array([-0.1, 0.9, -1.2]))[0]
t3 = np.array([[-1.0, -6.0, 28.0]], float).T

p1 = np.array([[853.0, 656.0]]).T
p2 = np.array([[814.0, 655.0]]).T
p3 = np.array([[798.0, 535.0]]).T


In [20]:
T1 = np.concatenate((
    np.concatenate((R1, t1), axis=1),
    np.array([[0,0,0,1]])
))

T2 = np.concatenate((
    np.concatenate((R2, t2), axis=1),
    np.array([[0,0,0,1]])
))

T = T1 @ np.linalg.inv(T2)
R = T[:3, :3]
t = T[:3,3][:, None]

E = CrossOp(t[:,0]) @ R
F = np.linalg.inv(K).T @ E @ np.linalg.inv(K)
l = F @ InvPi(p2)
shortest_distance_to_line(InvPi(p1), l)


array([[160.73201324]])

## excercise 16

In [21]:
triangulate_nonlin(
    (p1, p2, p3),
    (
        projectpoints(K, R1, t1),
        projectpoints(K, R2, t2),
        projectpoints(K, R3, t3)
    )
)

array([[ 0.25475577],
       [-1.53923692],
       [-0.14899204]])

## excercise 17

In [22]:
R = cv2.Rodrigues(np.array([-1.9, 0.1, -0.2]))[0]
t = np.array([[-1.7, 1.3, 1.5]], float).T

cam1_pos = -R.T @ t
prettyprint(cam1_pos, 1)

[[ 1.8]
 [ 1.9]
 [-0.4]]


## excercise 18

In [23]:
K = np.array([[300, 0, 840], [0, 300, 620.0], [0, 0, 1]], float)
R1 = cv2.Rodrigues(np.array([-2.3, -0.7, 1.0]))[0]
t1 = np.array([[0.0, -1.0, 4.0]], float).T
R2 = cv2.Rodrigues(np.array([-0.6, 0.5, -0.9]))[0]
t2 = np.array([[0.0, 0.0, 9.0]], float).T
R3 = cv2.Rodrigues(np.array([-0.1, 0.9, -1.2]))[0]
t3 = np.array([[-1.0, -6.0, 28.0]], float).T

p1 = np.array([[853.0, 656.0]]).T
p2 = np.array([[814.0, 655.0]]).T
p3 = np.array([[798.0, 535.0]]).T

Q = np.array([[-0.38], [0.1], [1.32]])


In [24]:
T2 = np.concatenate((
    np.concatenate((R2, t2), axis=1),
    np.array([[0,0,0,1]])
))

T3 = np.concatenate((
    np.concatenate((R3, t3), axis=1),
    np.array([[0,0,0,1]])
))

Pi(T2 @ np.linalg.inv(T3) @ InvPi(Q))

array([[  2.1880602 ],
       [ -5.26185229],
       [-17.77577456]])

## excercise 19

In [25]:
n = 4
eps_hat = 1 - 465 / 1177
p = 0.9
np.log(1-p) / np.log(1 - (1 - eps_hat) ** n)

93.36093153241097

# Part 20

## excercise 20

In [26]:
sigma=1.4
3.84*sigma**2

7.526399999999999

## excercise 21

In [27]:
primary = np.array([12., 9, 10, 13, 18, 25, 33, 40, 46, 49, 48, 45, 39, 31, 23, 17])
secondary = np.array([15., 29, 43, 49, 43, 29, 15, 10])
n_1 = 40

fft_primary = np.fft.rfft(primary, axis=0)
theta_primary = np.angle(fft_primary[1])

fft_secondary = np.fft.rfft(secondary, axis=0)
theta_secondary = np.angle(fft_secondary[1])

# heterodyne principle
theta_c = theta_primary - theta_secondary
theta_c %= 2*np.pi

o_primary = np.round((n_1 * theta_c - theta_primary) / (2 * np.pi))
theta_est = (2 * np.pi * o_primary + theta_primary) / n_1
theta_est

4.934396234333328

## excercise 22